In [1]:
pip uninstall -y tensorflow tf-keras tensorboard

Found existing installation: tensorflow 2.20.0
Uninstalling tensorflow-2.20.0:
  Successfully uninstalled tensorflow-2.20.0
Found existing installation: tf_keras 2.20.0
Uninstalling tf_keras-2.20.0:
  Successfully uninstalled tf_keras-2.20.0
Found existing installation: tensorboard 2.20.0
Uninstalling tensorboard-2.20.0:
  Successfully uninstalled tensorboard-2.20.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch; print(torch.cuda.is_available())  # should be True


False


In [2]:
# Upgrade pip
!pip install --upgrade pip

# Install tensorflow and keras 
!pip install --no-cache-dir tensorflow==2.20.0 tf-keras==2.20.0

# Install torch and torchdata
!pip install --no-deps torch torchdata --quiet

!pip install -U \
    datasets \
    transformers \
    accelerate \
    evaluate \
    bert-score \
    peft --quiet

   ---------------------------------------- 0.0/332.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/332.0 MB ? eta -:--:--
   ---------------------------------------- 0.8/332.0 MB 3.3 MB/s eta 0:01:40
   ---------------------------------------- 1.3/332.0 MB 3.0 MB/s eta 0:01:50
   ---------------------------------------- 1.8/332.0 MB 2.8 MB/s eta 0:01:57
   ---------------------------------------- 2.4/332.0 MB 2.7 MB/s eta 0:02:02
   ---------------------------------------- 2.6/332.0 MB 2.7 MB/s eta 0:02:03
   ---------------------------------------- 3.1/332.0 MB 2.5 MB/s eta 0:02:12
   ---------------------------------------- 3.4/332.0 MB 2.5 MB/s eta 0:02:14
   ---------------------------------------- 3.7/332.0 MB 2.3 MB/s eta 0:02:24
    --------------------------------------- 4.2/332.0 MB 2.2 MB/s eta 0:02:32
    --------------------------------------- 4.5/332.0 MB 2.1 MB/s eta 0:02:37
    --------------------------------------- 4.7/332.0 MB 2.0 MB/s eta 0:02:48


Reason for being yanked: <none given>


In [1]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer, BitsAndBytesConfig
import torch
import time
import evaluate
import pandas as pd
import numpy as np
import json

ImportError: cannot import name 'InterpolationMode' from 'torchvision.transforms' (C:\Users\chama\anaconda3\Lib\site-packages\torchvision\transforms\__init__.py)

In [4]:
train_dataset = []
test_dataset = []

with open('final_dataset.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        if line.strip():  # skip empty lines
            train_dataset.append(json.loads(line))

print(f"Loaded {len(train_dataset)} records.")
print(train_dataset[0]) 

Loaded 13926 records.
{'question': 'What is the primary subject of this image?', 'answer': 'This is a background image without a plant leaf.', 'split': 'train'}


In [5]:
with open('test.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        if line.strip():  # skip empty lines
            test_dataset.append(json.loads(line))

print(f"Loaded {len(test_dataset)} records.")
print(test_dataset[0])

Loaded 474 records.
{'question': "Can you trace the cause of this vegetation's appearance?", 'answer': 'The mosaic pattern of light and dark green areas is a classic symptom of ToMV.', 'split': 'test'}


In [6]:
with open('test.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        if line.strip():  # skip empty lines
            test_dataset.append(json.loads(line))

print(f"Loaded {len(test_dataset)} records.")
print(test_dataset[0])

Loaded 948 records.
{'question': "Can you trace the cause of this vegetation's appearance?", 'answer': 'The mosaic pattern of light and dark green areas is a classic symptom of ToMV.', 'split': 'test'}


In [7]:
model_name = "Qwen/Qwen2.5-1.5B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map="auto")

`torch_dtype` is deprecated! Use `dtype` instead!


In [8]:
prompt = "Explain how CNNs help in plant disease detection."
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
out = model.generate(**inputs, max_new_tokens=150)
print(tokenizer.decode(out[0], skip_special_tokens=True))

Explain how CNNs help in plant disease detection. Convolutional Neural Networks (CNNs) are powerful tools for image recognition and classification, making them particularly useful for detecting diseases on plants. Here’s a detailed explanation of how they contribute to this process:

### 1. **Feature Extraction**
   - **Image Processing**: CNNs start by processing the input image through multiple layers designed to extract features from raw pixel data.
   - **Convolutional Layers**: These layers apply filters that slide over the entire image, identifying edges, textures, and patterns. The number of these filters is determined by the network architecture.
   - **Pooling Layers**: After feature extraction, pooling layers reduce the spatial dimensions while retaining essential information. This helps in reducing the computational complexity and noise.

### 2. **


In [8]:
from datasets import Dataset

MAX_LEN = 512  # shorter than 1024 for speed and VRAM

train_ds = Dataset.from_list(train_dataset)
test_ds  = Dataset.from_list(test_dataset)

def tokenize_function(batch):
    start = "Answer Following Question\n\n"
    end = "\n\nAnswer: "
    prompts = [f"{start}{q}{end}" for q in batch["question"]]
    texts = [p + a for p, a in zip(prompts, batch["answer"])]
    tok = tokenizer(texts, truncation=True, max_length=MAX_LEN, padding=False)

    # labels = input_ids with prompt part masked to -100
    labels = []
    for p, ids in zip(prompts, tok["input_ids"]):
        p_len = len(tokenizer(p, add_special_tokens=False)["input_ids"])
        lab = [-100]*p_len + ids[p_len:]
        labels.append(lab[:len(ids)])
    tok["labels"] = labels
    return tok

tokenized_train = train_ds.map(tokenize_function, batched=True, batch_size=256, remove_columns=train_ds.column_names)
tokenized_test  = test_ds.map(tokenize_function,  batched=True, batch_size=256, remove_columns=test_ds.column_names)

from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

Map:   0%|          | 0/13926 [00:00<?, ? examples/s]

Map:   0%|          | 0/948 [00:00<?, ? examples/s]

In [9]:
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_train.shape}")
print(f"Training: {tokenized_test.shape}")

print("tokenzined train: ",tokenized_train)
print("tokenzined test: ",tokenized_test)

Shapes of the datasets:
Training: (13926, 3)
Training: (948, 3)
tokenzined train:  Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 13926
})
tokenzined test:  Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 948
})


## LoRA Fine-Tune

In [10]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=32,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],   
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM        
)

### Number of trainable parameters

In [11]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(model))

trainable model parameters: 1543714304
all model parameters: 1543714304
percentage of trainable model parameters: 100.00%


### Add LoRA model

In [12]:
peft_model = get_peft_model(model, 
                            lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 4358144
all model parameters: 1548072448
percentage of trainable model parameters: 0.28%


### PEFT Adpoter Train

In [13]:
from transformers import TrainingArguments, Trainer, TrainerCallback

training_args = TrainingArguments(
    output_dir="./peft-qwen-training",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    num_train_epochs=1,
    fp16=True,
    bf16=False,
    logging_steps=50,          # print every 50 steps
    report_to="none",
    dataloader_num_workers=2,
    optim="adamw_torch"
)
class LossPrinter(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs and "loss" in logs:
            print(f"[step {state.global_step}] train_loss = {logs['loss']:.4f}")

peft_trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=None,            
    data_collator=data_collator,
)


peft_trainer.add_callback(LossPrinter())

The model is already on multiple devices. Skipping the move to device specified in `args`.


In [14]:
pip install --index-url https://download.pytorch.org/whl/cu121 torch torchvision

Looking in indexes: https://download.pytorch.org/whl/cu121
Note: you may need to restart the kernel to use updated packages.


In [15]:
import torch
print("CUDA:", torch.cuda.is_available())
print("Any CPU offload in map?:", any(str(d).startswith("cpu") for d in getattr(peft_trainer.model, "hf_device_map", {}).values()) if hasattr(peft_trainer.model, "hf_device_map") else False)


CUDA: False
Any CPU offload in map?: True


In [16]:
pip install --upgrade --index-url https://download.pytorch.org/whl/cu121 torch torchvision


Looking in indexes: https://download.pytorch.org/whl/cu121
Note: you may need to restart the kernel to use updated packages.


In [23]:
peft_trainer.train()

peft_model_path="./safelora-checkpoint-local"

peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

AssertionError: Torch not compiled with CUDA enabled